In [1]:
!pip install langid
!pip install -U sentence-transformers
!pip install fasttext


     |████████████████████████████████| 1.9MB 2.8MB/s 
  Created wheel for langid: filename=langid-1.1.6-cp36-none-any.whl size=1941190 sha256=f56ae8a9a101855d73b7c69a3f2ae13d58d3371385fec9a0b5534b83d7d22564
  Stored in directory: /root/.cache/pip/wheels/29/bc/61/50a93be85d1afe9436c3dc61f38da8ad7b637a38af4824e86e
Successfully built langid
     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 368kB 3.7MB/s 
     |████████████████████████████████| 1.0MB 6.1MB/s 
     |████████████████████████████████| 870kB 8.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.4.1-cp36-none-any.whl size=61094 sha256=0217b04e9a61079c914ac21c1bda2272a491766321c62f3ca7e59422202665b4
  Stored in directory: /root/.cache/pip/wheels/12/a5/1c/03b7d87e027121fe1e23048007594e73f39a23e833658529c7
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=98afaaa035f4e933d530b126e5972dc43e08d25f6d951a9585642e

In [2]:
!git clone https://github.com/LucaBassanese/Tesi

Cloning into 'Tesi'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 31 (delta 8), reused 15 (delta 2), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy


import matplotlib.pyplot as plt

#Keywords
import sklearn

#wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from nltk.corpus import stopwords

#transalte
import langid

import random


#cluster of papers
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

#cluster hierarchical
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import ward, dendrogram

#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
pd.options.display.max_rows = 150

#SBERT
from sentence_transformers import SentenceTransformer


from nltk.util import ngrams

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Titolo e Abstract

In [0]:
url =  'https://raw.githubusercontent.com/LucaBassanese/Tesi/master/data/abstract.xlsx'
df = pd.read_excel(url)
dft = df.copy()
df = df[df['Abstract inglese'].notnull()]
df = df.drop_duplicates(['Abstract inglese'])
df['titabs']=  df.Titolo + ' ' + df['Abstract inglese']

lingua = [langid.classify(testo)[0] for testo in df.Titolo]
lingua = pd.Series(lingua)
df = df[(lingua == 'en').values]

#tokenizer che toglie la punteggiatura
tokenizer = RegexpTokenizer(r'\w+')


In [0]:
#tokenizzo togliendo la punteggiatura
words_ta = [tokenizer.tokenize(fr.lower()) for fr in df.titabs] 
#creo bigrammi e trigrammi
bigrams_t = [list(ngrams(word,2)) for word in words_ta ]
bigrams_ta = [['_'.join(list(w)) for w in bi] for bi in bigrams_t]
trigrams_t = [list(ngrams(word,3)) for word in words_ta ]
trigrams_ta = [['_'.join(list(w)) for w in tri] for tri in trigrams_t]

In [0]:
#creo i tagged document per il doc2vec
tagged_data_ta = [TaggedDocument(words=words_ta[i], tags=[str(i)]) for i in range(len(df))]
tagged_bi_ta = [TaggedDocument(words=bigrams_ta[i], tags=[str(i)]) for i in range(len(df))]
tagged_tri_ta = [TaggedDocument(words=trigrams_ta[i], tags=[str(i)]) for i in range(len(df))]

# Titoli


In [0]:
dft = dft.drop_duplicates(['Titolo'])
# Seleziono sogli i titoli in inglese
lingua = [langid.classify(testo)[0] for testo in dft.Titolo]
lingua = pd.Series(lingua)
dft = dft[(lingua == 'en').values]

In [8]:
#tokenizzo togliendo la punteggiatura
words = [tokenizer.tokenize(fr.lower()) for fr in dft.Titolo ] 
#creo bigrammi e trigrammi
bigrams_t = [list(ngrams(word,2)) for word in words ]
bigrams = [['_'.join(list(w)) for w in bi] for bi in bigrams_t]
trigrams_t = [list(ngrams(word,3)) for word in words ]
trigrams = [['_'.join(list(w)) for w in tri] for tri in trigrams_t]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'ngrams' raised StopIteration
  """


In [0]:
#creo i tagged document per il doc2vec
tagged_data = [TaggedDocument(words=words[i], tags=[str(i)]) for i in range(len(dft))]
tagged_bi = [TaggedDocument(words=bigrams[i], tags=[str(i)]) for i in range(len(dft))]
tagged_tri = [TaggedDocument(words=trigrams[i], tags=[str(i)]) for i in range(len(dft))]


# Frasi unite

In [0]:
frasi = [' '.join(word) for word in words]
frasi_bi = [' '.join(bi) for bi in bigrams]
frasi_tri = [' '.join(tri) for tri in trigrams]
frasi_ta =  [' '.join(word) for word in words_ta]
frasi_bi_ta = [' '.join(bi) for bi in bigrams_ta]
frasi_tri_ta = [' '.join(tri) for tri in trigrams_ta]

# Distribuzione parole 


In [0]:
#creo lessico di tutte le parole
lessico = tokenizer.tokenize(' '.join(frasi ))
stop_words = set(stopwords.words('english')) 
lessico = [w for w in lessico if not w in stop_words] 
#creo la distribuzione delle parole nei titoli
distr = (pd.Series(lessico)).value_counts()

In [0]:
distr = distr[distr >2] 

In [0]:
parover95 = list((distr[distr > np.percentile(distr, 95)]).index)
par75_80 = list((distr[distr.between(np.percentile(distr, 75), np.percentile(distr, 80))]).index)
par50_55 = list((distr[distr.between(np.percentile(distr, 50), np.percentile(distr, 55))]).index)

In [0]:
parindoc_1 = [[d.count(wr) for wr in parover95]for d in frasi]
somma_1 = [sum(doc) for doc in parindoc_1]
ind_1 = sorted(range(len(somma_1)), key=lambda i: somma_1[i])[-10:]

In [0]:
parindoc_2 = [[d.count(wr) for wr in par75_80]for d in frasi]
somma_2 = [sum(doc) for doc in parindoc_2]
ind_2 = sorted(range(len(somma_2)), key=lambda i: somma_2[i])[-10:]

In [0]:
parindoc_3 = [[d.count(wr) for wr in par50_55]for d in frasi]
somma_3 = [sum(doc) for doc in parindoc_3]
ind_3 = sorted(range(len(somma_3)), key=lambda i: somma_3[i])[-11:]

In [0]:
indici = (pd.Series(ind_1+ ind_2 + ind_3)).drop_duplicates()


# Frasi to txt

In [0]:
pd.DataFrame(frasi, columns=['frasi']).to_csv('frasi.txt',  index=None, header = False)
pd.DataFrame(frasi_bi, columns=['frasi']).to_csv('frasi_bi.txt',  index=None, header = False)
pd.DataFrame(frasi_tri, columns=['frasi']).to_csv('frasi_tri.txt',  index=None, header = False)

In [21]:
frasi_tri

['oligopoly_games_with games_with_nonlinear with_nonlinear_demand nonlinear_demand_and demand_and_cost and_cost_functions cost_functions_two functions_two_boundedly two_boundedly_rational boundedly_rational_adjustment rational_adjustment_processes',
 'investment_decisions_and decisions_and_the and_the_soft the_soft_budget soft_budget_constraint',
 'modeling_and_forecasting and_forecasting_cointegrated forecasting_cointegrated_relationships cointegrated_relationships_among relationships_among_heavy among_heavy_oil heavy_oil_and oil_and_product and_product_prices',
 'oligopoly_games_with games_with_local with_local_monopolistic local_monopolistic_approximation',
 'rockets_and_feathers and_feathers_revisited feathers_revisited_an revisited_an_international an_international_comparison international_comparison_on comparison_on_european on_european_gasoline european_gasoline_markets',
 'agricultural_technology_adoption technology_adoption_and adoption_and_poverty and_poverty_reduction povert

In [0]:
!./fasttext sent2vec -input frasi.txt -output my_model -minCount 1 -dim 700 -epoch 9 -lr 0.2 -wordNgrams 2 -loss ns -neg 10 -thread 20 -t 0.000005 -dropoutK 4 -minCountLabel 20 -bucket 4000000 -maxVocabSize 750000 -numCheckPoints 10

/bin/bash: ./fasttext: No such file or directory


# SBERT pre training

In [0]:
gensim.models??

# Sent2vec

In [0]:
import fasttext

ModuleNotFoundError: ignored

# Doc2vec


In [0]:
# esempio 
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha


## Data dm


In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

In [0]:
model.save('doc2vec_uni_dm_300')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [0]:
model.save('doc2vec_uni_dm_400')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_uni_dm_500')

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL



## Data dbow

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_uni_dbow_300')

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_uni_dbow_400')

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_uni_dbow_500')

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL



## Bigram dm

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_bi_dm_300')

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_bi_dm_400')

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_bi_dm_500')

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 2min 35s, sys: 2.71 s, total: 2min 37s
Wall time: 1min 43s


## Bigram dbow

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_bi_dbow_300')

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 1min 23s, sys: 6.65 s, total: 1min 29s
Wall time: 1min 9s


In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_bi_dbow_400')

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_bi_dbow_500')

## Trigram dm

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_tri_dm_300')

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_tri_dm_400')

In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_tri_dm_500')

## Trigram dbow

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_tri_dbow_300')

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_tri_dbow_400')

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 1min 58s, sys: 7.8 s, total: 2min 6s
Wall time: 1min 44s


In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save('doc2vec_tri_dbow_500')


## Load doc2vec

In [0]:
model2 = Doc2Vec.load('ciao')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

# SBERT

In [0]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(frasi)
sbert = np.stack( sentence_embeddings, axis=0 )

100%|██████████| 405M/405M [00:24<00:00, 16.4MB/s]


In [0]:
model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

100%|██████████| 1.24G/1.24G [00:18<00:00, 67.1MB/s]


In [0]:
import pandas as pd
pd.read_csv??

In [0]:
sentence_embeddings = model.encode(frasi)
sbert = np.stack( sentence_embeddings, axis=0 )

In [0]:
sbert.shape

(1548, 1024)

In [0]:
sbert

array([[-1.1434323 , -0.18188004,  0.91623545, ..., -0.48097208,
        -0.9856035 ,  0.8776383 ],
       [-0.40389788, -0.41126215,  0.42457682, ...,  0.5315726 ,
        -0.87461996,  0.44353035],
       [-0.24677515, -0.12850131,  0.44034338, ..., -0.7237369 ,
        -0.31194127, -0.29219583],
       ...,
       [-0.3599878 ,  0.2995323 ,  0.62660736, ..., -0.2494359 ,
        -0.6457086 ,  0.389404  ],
       [-0.49856946,  0.05594404,  0.3238621 , ..., -0.1141041 ,
        -0.24745905,  0.60181206],
       [ 0.03410498, -0.08580752,  1.8610344 , ..., -1.1958299 ,
        -1.1618495 , -0.1159507 ]], dtype=float32)